# Predicción de crímenes
### Trabajo Fin de Máster para el Máster Universitario en Ciencia de Datos
### Universitat Oberta de Catalunya
### Realizado por Álvaro Pavón Díaz

## Parte 2: Modelado y evaluación de los datos
Durante este Jupyter Notebook nos dedicaremos a realizar el entrenamiento de los conjunto de datos sobre diferentes métodos.

Para ello lo primero se va a proceder a importar las diferentes clases que se necesitarán a lo largo de este notebook y la importación de los archivos que se va a utilizar. Además de mostrar que los proceso realizados a través de TensorFlow van a ser acelerados mediante GPU (GeForce RTX 2070 SUPER)

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time_function
import tensorflow as tf
#from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

#print(device_lib.list_local_devices())
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [2]:
crimes = pd.read_csv ('crimes_clean.csv', sep='""', delimiter=',', engine='python')
crimes.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3
1,Property Lost,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30
2,Other,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3
4,Aircraft,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3


In [3]:
crimes_time = pd.read_csv ('crimes_clean_time_2.csv', sep='""', delimiter=',', engine='python')
crimes_time.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH,TIME_X,TIME_Y,Lat_M,Long_M
0,Desorden,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3,0.518198,0.000331,0.185665,0.267386
1,Otros,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30,0.239106,0.073462,0.736023,0.201687
2,Otros,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3,0.517840,0.000318,0.465558,0.473233
3,Desorden,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3,0.518198,0.000331,0.781170,0.553614
4,Otros,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3,0.518198,0.000331,0.878350,0.716335


In [4]:
crimes_time=crimes_time[['OFFENSE_CODE_GROUP', 'SHOOTING','YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','HOUR','IS_NIGHT','Lat', 'Lat_M', 'Long','Long_M', 'TIME_X', 'TIME_Y', 'DISTRICT']]
crimes_time.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
0,Desorden,0.0,2018,10,3,Wednesday,20,1,42.262608,0.185665,-71.121186,0.267386,0.518198,0.000331,E18
1,Otros,0.0,2018,8,30,Thursday,20,1,42.352111,0.736023,-71.135311,0.201687,0.239106,0.073462,D14
2,Otros,0.0,2018,10,3,Wednesday,19,1,42.308126,0.465558,-71.076930,0.473233,0.517840,0.000318,B2
3,Desorden,0.0,2018,10,3,Wednesday,20,1,42.359454,0.781170,-71.059648,0.553614,0.518198,0.000331,A1
4,Otros,0.0,2018,10,3,Wednesday,20,1,42.375258,0.878350,-71.024663,0.716335,0.518198,0.000331,A7


In [5]:
crimes_times = crimes_time.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_times.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
127095,Sexual,0.0,2015,6,15,Monday,0,1,42.291093,0.360820,-71.065945,0.524324,0.022469,0.648202,C11
241630,Otros,0.0,2015,6,15,Monday,0,1,42.283634,0.314958,-71.082813,0.445868,0.022469,0.648202,B3
288821,Fraude,0.0,2015,6,15,Monday,0,1,42.360205,0.785788,-71.056208,0.569614,0.022469,0.648202,A1
295842,Otros,0.0,2015,6,15,Monday,0,1,42.293606,0.376273,-71.071887,0.496690,0.022469,0.648202,C11
300018,Hurto,0.0,2015,6,15,Monday,0,1,42.300217,0.416923,-71.080979,0.454397,0.022469,0.648202,B3


In [6]:
crimes_time_one = crimes_time
le_dw = LabelEncoder()
dw_labels = le_dw.fit_transform(crimes_time_one['DAY_OF_WEEK'])
crimes_time_one['DAY_OF_WEEK_label'] = dw_labels

one_dw = OneHotEncoder()
days_of_week = one_dw.fit_transform(crimes_time_one[['DAY_OF_WEEK_label']]).toarray()
days_of_week_labels = list(le_dw.classes_)
days_of_week_df = pd.DataFrame(days_of_week, columns=days_of_week_labels)



In [7]:
le_ocg = LabelEncoder()
ocg_labels = le_ocg.fit_transform(crimes_time_one['OFFENSE_CODE_GROUP'])
crimes_time_one['OFFENSE_CODE_label'] = ocg_labels

one_ocg = OneHotEncoder()
ocg = one_ocg.fit_transform(crimes_time_one[['OFFENSE_CODE_label']]).toarray()
ocg_labels = list(le_ocg.classes_)
ocg_df = pd.DataFrame(ocg, columns=ocg_labels)

In [8]:
#le_year = LabelEncoder()
#year_labels = le_year.fit_transform(crimes_time_one['YEAR'])
#crimes_time_one['year_label'] = year_labels

#one_year = OneHotEncoder()
#year = one_year.fit_transform(crimes_time_one[['year_label']]).toarray()
#year_labels = list(le_year.classes_)
#year_df = pd.DataFrame(year, columns=year_labels)

In [9]:
le_d = LabelEncoder()
d_labels = le_d.fit_transform(crimes_time_one['DISTRICT'])
crimes_time_one['DISTRICT_labels'] = d_labels

In [10]:
crimes_time_one = pd.concat([ocg_df,days_of_week_df, crimes_time_one], axis=1)

crimes_time_one.drop('DAY_OF_WEEK_label', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_label', 1, inplace= True)
crimes_time_one.drop('DAY_OF_WEEK', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_GROUP', 1, inplace= True)
#crimes_time_one.drop('year_label', 1, inplace= True)
crimes_time_one.drop('DISTRICT', 1, inplace= True)
crimes_time_one.drop('Lat', 1, inplace= True)
crimes_time_one.drop('Long', 1, inplace= True)
crimes_time_one.drop('Lat_M', 1, inplace= True)
crimes_time_one.drop('Long_M', 1, inplace= True)

In [11]:
crimes_time_one = crimes_time_one.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_time_train_test = crimes_time_one.head(int(crimes_time_one.shape[0] * 0.8))
crimes_time_validation = crimes_time_one.tail(crimes_time_one.shape[0] - int(crimes_times.shape[0] * 0.8))
crimes_time_train_test.drop('YEAR', 1, inplace= True)
crimes_time_validation.drop('YEAR', 1, inplace= True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
crimes_time_train_test.head(5)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Tuesday,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels
127095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,5
241630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,4
288821,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,0
295842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,5
300018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,4


In [13]:
crimes_time_validation.head(5)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Tuesday,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels
61407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,14,0,0.91873,0.773249,4
61408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2,3,14,0,0.91873,0.773249,7
61411,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,14,0,0.91873,0.773249,8
61412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,14,0,0.91873,0.773249,8
61413,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,14,0,0.91873,0.773249,8


In [14]:
crimes_time_validation.shape

(60846, 31)

In [15]:
crimes_time_train_test.shape

(243383, 31)

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'Armas':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT_labels'], test_size = 0.25, random_state = 0)
#xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'OFFENSE_CODE_GROUP':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT'], test_size = 0.25, random_state = 0)

In [17]:
#xTrain.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
#xTest.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
xTrain.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
xTest.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)

In [18]:
xTrain.head(15)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,SHOOTING,IS_NIGHT,TIME_X,TIME_Y
231328,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.489538,0.999891
302995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.012670,0.611847
195984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.157651,0.135587
88225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.605984,0.011362
289842,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.108869,0.188526
82894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.232153,0.077794
212207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.017742,0.632013
102925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.168547,0.125649
196485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.121758,0.172994
290356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.097312,0.203618


In [20]:
xValidation = crimes_time_validation.loc[:,'Armas':'TIME_Y']
yValidation =  crimes_time_validation.loc[:,'DISTRICT_labels']
xValidation.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
def create_model (type_layer= 'LSTM', n_layers=1, n_embeding = 32,n_epochs = 20,batch_size=512, top_k=3):
    
    layer_r = tf.keras.layers.LSTM(n_embeding, return_sequences=True)
    layer_e = tf.keras.layers.LSTM(n_embeding)
    if type_layer == 'GRU':
        layer_r = tf.keras.layers.GRU(n_embeding, return_sequences=True)
        layer_e = tf.keras.layers.GRU(n_embeding)
        
    model_i = tf.keras.models.Sequential()
    model_i.add(tf.keras.layers.Embedding(30, n_embeding, input_length=27))
    if n_layers != 0:
        for i in range(n_layers):
            model_i.add(layer_r)
    
    model_i.add(layer_e)
    
    model_i.add(tf.keras.layers.Dense(12, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    metric= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
    model_i.compile (optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics=[metric] )
    
    model_i.fit(xTrain, yTrain, epochs = n_epochs,batch_size=batch_size, validation_data=(xTest, yTest))
    return model_i.evaluate(xValidation, yValidation)

In [21]:
create_model (type_layer= 'LSTM', n_layers=0, n_embeding = 128,n_epochs = 20,batch_size=256 )

Epoch 1/20
714/714 [==============================] - 48s 68ms/step - loss: 2.3587 - sparse_top_k_categorical_accuracy: 0.4227 - val_loss: 2.3537 - val_sparse_top_k_categorical_accuracy: 0.4260
Epoch 2/20
714/714 [==============================] - 48s 68ms/step - loss: 2.3547 - sparse_top_k_categorical_accuracy: 0.4246 - val_loss: 2.3543 - val_sparse_top_k_categorical_accuracy: 0.4226
Epoch 3/20
714/714 [==============================] - 47s 66ms/step - loss: 2.3543 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3537 - val_sparse_top_k_categorical_accuracy: 0.4265
Epoch 4/20
714/714 [==============================] - 48s 68ms/step - loss: 2.3536 - sparse_top_k_categorical_accuracy: 0.4255 - val_loss: 2.3525 - val_sparse_top_k_categorical_accuracy: 0.4265
Epoch 5/20
714/714 [==============================] - 50s 71ms/step - loss: 2.3532 - sparse_top_k_categorical_accuracy: 0.4257 - val_loss: 2.3518 - val_sparse_top_k_categorical_accuracy: 0.4307
Epoch 6/20
714/714 [==========

[2.3581621646881104, 0.41670775413513184]

In [22]:

def grid_search(n_layers=[0,1,2,3], type_layers= ['GRU','LSTM'], n_embedings=[64,128], n_epochs = 20, top_ks=[3,4]):
    i=0
    lista = {}
    for n_layer in n_layers:
        for type_l in type_layers:
            for n_embeding in n_embedings:
                for top_k in top_ks:
                    print("************ NUEVA PRUEBA *************")
                    print (type_l)
                    print (n_layer)
                    print (n_embeding)
                    print (top_k)
                    try:
                        fit = create_model (type_layer= type_l, n_layers=n_layer, n_embeding = n_embeding, n_epochs=n_epochs)
                        print(fit)
                        print(i)
                        lista[str(i)] = "type_l=" + type_l + ",n_layer="+ str(n_layer) + ",n_embeding" + str(n_embeding) + ",top_k=" + str(top_k)
                        i=i+1
                    except:
                        print("Imposible entrenar el modelos con los parametros siguientes")
    return lista

In [23]:
resultados0 = grid_search (n_layers=[0],type_layers= ['GRU','LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
GRU
0
64
3
Epoch 1/20
357/357 [==============================] - 18s 51ms/step - loss: 2.3575 - sparse_top_k_categorical_accuracy: 0.4220 - val_loss: 2.3539 - val_sparse_top_k_categorical_accuracy: 0.4260
Epoch 2/20
357/357 [==============================] - 17s 48ms/step - loss: 2.3547 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3531 - val_sparse_top_k_categorical_accuracy: 0.4303
Epoch 3/20
357/357 [==============================] - 17s 48ms/step - loss: 2.3543 - sparse_top_k_categorical_accuracy: 0.4257 - val_loss: 2.3542 - val_sparse_top_k_categorical_accuracy: 0.4265
Epoch 4/20
357/357 [==============================] - 17s 49ms/step - loss: 2.3470 - sparse_top_k_categorical_accuracy: 0.4351 - val_loss: 2.3448 - val_sparse_top_k_categorical_accuracy: 0.4398
Epoch 5/20
357/357 [==============================] - 17s 48ms/step - loss: 2.3373 - sparse_top_k_categorical_accuracy: 0.4459 - val_loss: 2.3334 - val_sparse_top_k_categori

Epoch 1/20
357/357 [==============================] - 41s 115ms/step - loss: 2.3577 - sparse_top_k_categorical_accuracy: 0.4216 - val_loss: 2.3546 - val_sparse_top_k_categorical_accuracy: 0.4292
Epoch 2/20
357/357 [==============================] - 40s 113ms/step - loss: 2.3553 - sparse_top_k_categorical_accuracy: 0.4252 - val_loss: 2.3552 - val_sparse_top_k_categorical_accuracy: 0.4262
Epoch 3/20
357/357 [==============================] - 41s 114ms/step - loss: 2.3551 - sparse_top_k_categorical_accuracy: 0.4253 - val_loss: 2.3545 - val_sparse_top_k_categorical_accuracy: 0.4302
Epoch 4/20
357/357 [==============================] - 42s 118ms/step - loss: 2.3477 - sparse_top_k_categorical_accuracy: 0.4349 - val_loss: 2.3351 - val_sparse_top_k_categorical_accuracy: 0.4499
Epoch 5/20
357/357 [==============================] - 42s 116ms/step - loss: 2.3359 - sparse_top_k_categorical_accuracy: 0.4464 - val_loss: 2.3346 - val_sparse_top_k_categorical_accuracy: 0.4466
Epoch 6/20
357/357 [=====

Epoch 1/20
357/357 [==============================] - 20s 56ms/step - loss: 2.3568 - sparse_top_k_categorical_accuracy: 0.4239 - val_loss: 2.3542 - val_sparse_top_k_categorical_accuracy: 0.4274
Epoch 2/20
357/357 [==============================] - 19s 54ms/step - loss: 2.3546 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3531 - val_sparse_top_k_categorical_accuracy: 0.4261
Epoch 3/20
357/357 [==============================] - 19s 53ms/step - loss: 2.3543 - sparse_top_k_categorical_accuracy: 0.4245 - val_loss: 2.3529 - val_sparse_top_k_categorical_accuracy: 0.4255
Epoch 4/20
357/357 [==============================] - 19s 54ms/step - loss: 2.3542 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3525 - val_sparse_top_k_categorical_accuracy: 0.4261
Epoch 5/20
357/357 [==============================] - 19s 53ms/step - loss: 2.3540 - sparse_top_k_categorical_accuracy: 0.4247 - val_loss: 2.3525 - val_sparse_top_k_categorical_accuracy: 0.4259
Epoch 6/20
357/357 [==========

Epoch 1/20
357/357 [==============================] - 57s 159ms/step - loss: 2.3603 - sparse_top_k_categorical_accuracy: 0.4214 - val_loss: 2.3561 - val_sparse_top_k_categorical_accuracy: 0.4161
Epoch 2/20
357/357 [==============================] - 56s 158ms/step - loss: 2.3554 - sparse_top_k_categorical_accuracy: 0.4243 - val_loss: 2.3540 - val_sparse_top_k_categorical_accuracy: 0.4228
Epoch 3/20
357/357 [==============================] - 57s 159ms/step - loss: 2.3548 - sparse_top_k_categorical_accuracy: 0.4248 - val_loss: 2.3545 - val_sparse_top_k_categorical_accuracy: 0.4258
Epoch 4/20
357/357 [==============================] - 57s 161ms/step - loss: 2.3542 - sparse_top_k_categorical_accuracy: 0.4243 - val_loss: 2.3528 - val_sparse_top_k_categorical_accuracy: 0.4275
Epoch 5/20
357/357 [==============================] - 57s 159ms/step - loss: 2.3539 - sparse_top_k_categorical_accuracy: 0.4243 - val_loss: 2.3535 - val_sparse_top_k_categorical_accuracy: 0.4256
Epoch 6/20
357/357 [=====

In [24]:
resultados1 = grid_search (n_layers=[1],type_layers= ['LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
LSTM
1
64
3
Epoch 1/20
357/357 [==============================] - 43s 121ms/step - loss: 2.3547 - sparse_top_k_categorical_accuracy: 0.4269 - val_loss: 2.3395 - val_sparse_top_k_categorical_accuracy: 0.4453
Epoch 2/20
357/357 [==============================] - 44s 122ms/step - loss: 2.3387 - sparse_top_k_categorical_accuracy: 0.4433 - val_loss: 2.3355 - val_sparse_top_k_categorical_accuracy: 0.4507
Epoch 3/20
357/357 [==============================] - 45s 127ms/step - loss: 2.3503 - sparse_top_k_categorical_accuracy: 0.4298 - val_loss: 2.3531 - val_sparse_top_k_categorical_accuracy: 0.4258
Epoch 4/20
357/357 [==============================] - 43s 121ms/step - loss: 2.3539 - sparse_top_k_categorical_accuracy: 0.4258 - val_loss: 2.3530 - val_sparse_top_k_categorical_accuracy: 0.4293
Epoch 5/20
357/357 [==============================] - 42s 119ms/step - loss: 2.3537 - sparse_top_k_categorical_accuracy: 0.4263 - val_loss: 2.3528 - val_sparse_top_k_ca

1902/1902 [==============================] - 11s 6ms/step - loss: 2.3280 - sparse_top_k_categorical_accuracy: 0.4454
[2.328011989593506, 0.44537028670310974]
1
************ NUEVA PRUEBA *************
LSTM
1
128
3
Epoch 1/20
357/357 [==============================] - 122s 342ms/step - loss: 2.3618 - sparse_top_k_categorical_accuracy: 0.4193 - val_loss: 2.3547 - val_sparse_top_k_categorical_accuracy: 0.4261
Epoch 2/20
357/357 [==============================] - 124s 346ms/step - loss: 2.3553 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3534 - val_sparse_top_k_categorical_accuracy: 0.4269
Epoch 3/20
357/357 [==============================] - 122s 341ms/step - loss: 2.3546 - sparse_top_k_categorical_accuracy: 0.4246 - val_loss: 2.3529 - val_sparse_top_k_categorical_accuracy: 0.4266
Epoch 4/20
357/357 [==============================] - 124s 346ms/step - loss: 2.3542 - sparse_top_k_categorical_accuracy: 0.4251 - val_loss: 2.3526 - val_sparse_top_k_categorical_accuracy: 0.4258
Epo

In [25]:
n_embeding=20

model_wo_D = tf.keras.models.Sequential()
model_wo_D.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model_wo_D.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D] )
model_fitted_wo_D = model_wo_D.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D = model_wo_D.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3321 - sparse_top_k_categorical_accuracy: 0.4504 - val_loss: 2.3252 - val_sparse_top_k_categorical_accuracy: 0.4539
Epoch 2/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3256 - sparse_top_k_categorical_accuracy: 0.4533 - val_loss: 2.3251 - val_sparse_top_k_categorical_accuracy: 0.4533
Epoch 3/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3238 - sparse_top_k_categorical_accuracy: 0.4540 - val_loss: 2.3242 - val_sparse_top_k_categorical_accuracy: 0.4560
Epoch 4/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3225 - sparse_top_k_categorical_accuracy: 0.4551 - val_loss: 2.3241 - val_sparse_top_k_categorical_accuracy: 0.4564
Epoch 5/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3215 - sparse_top_k_categorical_accuracy: 0.4552 - val_loss: 2.3237 - val_sparse_top_k_categorical_accuracy: 0.4548
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2827 - sparse_top_k_categorical_accuracy: 0.4722 - val_loss: 2.3474 - val_sparse_top_k_categorical_accuracy: 0.4442
Epoch 44/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2819 - sparse_top_k_categorical_accuracy: 0.4734 - val_loss: 2.3455 - val_sparse_top_k_categorical_accuracy: 0.4449
Epoch 45/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2813 - sparse_top_k_categorical_accuracy: 0.4732 - val_loss: 2.3455 - val_sparse_top_k_categorical_accuracy: 0.4457
Epoch 46/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2805 - sparse_top_k_categorical_accuracy: 0.4739 - val_loss: 2.3490 - val_sparse_top_k_categorical_accuracy: 0.4460
Epoch 47/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2801 - sparse_top_k_categorical_accuracy: 0.4745 - val_loss: 2.3515 - val_sparse_top_k_categorical_accuracy: 0.4455
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2596 - sparse_top_k_categorical_accuracy: 0.4833 - val_loss: 2.3732 - val_sparse_top_k_categorical_accuracy: 0.4411
Epoch 86/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2591 - sparse_top_k_categorical_accuracy: 0.4827 - val_loss: 2.3740 - val_sparse_top_k_categorical_accuracy: 0.4396
Epoch 87/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2585 - sparse_top_k_categorical_accuracy: 0.4836 - val_loss: 2.3768 - val_sparse_top_k_categorical_accuracy: 0.4366
Epoch 88/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2583 - sparse_top_k_categorical_accuracy: 0.4832 - val_loss: 2.3739 - val_sparse_top_k_categorical_accuracy: 0.4402
Epoch 89/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2579 - sparse_top_k_categorical_accuracy: 0.4839 - val_loss: 2.3753 - val_sparse_top_k_categorical_accuracy: 0.4383
Epoch 90/100
1427/14

In [26]:
n_embeding=20

model_wD = tf.keras.models.Sequential()
model_wD.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model_wD.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD.add(tf.keras.layers.Dropout(0.25))
model_wD.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD] )
model_fitted_wD = model_wD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD = model_wD.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3353 - sparse_top_k_categorical_accuracy: 0.4477 - val_loss: 2.3258 - val_sparse_top_k_categorical_accuracy: 0.4555
Epoch 2/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3270 - sparse_top_k_categorical_accuracy: 0.4533 - val_loss: 2.3268 - val_sparse_top_k_categorical_accuracy: 0.4526
Epoch 3/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3248 - sparse_top_k_categorical_accuracy: 0.4543 - val_loss: 2.3254 - val_sparse_top_k_categorical_accuracy: 0.4534
Epoch 4/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3241 - sparse_top_k_categorical_accuracy: 0.4545 - val_loss: 2.3229 - val_sparse_top_k_categorical_accuracy: 0.4549
Epoch 5/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3226 - sparse_top_k_categorical_accuracy: 0.4551 - val_loss: 2.3235 - val_sparse_top_k_categorical_accuracy: 0.4545
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2972 - sparse_top_k_categorical_accuracy: 0.4667 - val_loss: 2.3342 - val_sparse_top_k_categorical_accuracy: 0.4504
Epoch 44/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2969 - sparse_top_k_categorical_accuracy: 0.4667 - val_loss: 2.3354 - val_sparse_top_k_categorical_accuracy: 0.4493
Epoch 45/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2967 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.3341 - val_sparse_top_k_categorical_accuracy: 0.4504
Epoch 46/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2963 - sparse_top_k_categorical_accuracy: 0.4669 - val_loss: 2.3334 - val_sparse_top_k_categorical_accuracy: 0.4510
Epoch 47/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2958 - sparse_top_k_categorical_accuracy: 0.4668 - val_loss: 2.3347 - val_sparse_top_k_categorical_accuracy: 0.4499
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2846 - sparse_top_k_categorical_accuracy: 0.4714 - val_loss: 2.3456 - val_sparse_top_k_categorical_accuracy: 0.4446
Epoch 86/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2841 - sparse_top_k_categorical_accuracy: 0.4723 - val_loss: 2.3450 - val_sparse_top_k_categorical_accuracy: 0.4468
Epoch 87/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2838 - sparse_top_k_categorical_accuracy: 0.4722 - val_loss: 2.3480 - val_sparse_top_k_categorical_accuracy: 0.4450
Epoch 88/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2839 - sparse_top_k_categorical_accuracy: 0.4712 - val_loss: 2.3447 - val_sparse_top_k_categorical_accuracy: 0.4471
Epoch 89/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2835 - sparse_top_k_categorical_accuracy: 0.4725 - val_loss: 2.3453 - val_sparse_top_k_categorical_accuracy: 0.4479
Epoch 90/100
1427/14

In [27]:
n_embeding=20

model_wDD = tf.keras.models.Sequential()
model_wDD.add(tf.keras.layers.Dense(512, activation='relu', input_dim=27))
model_wDD.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD] )
model_fitted_wDD = model_wDD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD = model_wDD.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3369 - sparse_top_k_categorical_accuracy: 0.4479 - val_loss: 2.3304 - val_sparse_top_k_categorical_accuracy: 0.4485
Epoch 2/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3286 - sparse_top_k_categorical_accuracy: 0.4530 - val_loss: 2.3248 - val_sparse_top_k_categorical_accuracy: 0.4541
Epoch 3/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3269 - sparse_top_k_categorical_accuracy: 0.4532 - val_loss: 2.3251 - val_sparse_top_k_categorical_accuracy: 0.4536
Epoch 4/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3256 - sparse_top_k_categorical_accuracy: 0.4529 - val_loss: 2.3237 - val_sparse_top_k_categorical_accuracy: 0.4548
Epoch 5/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3249 - sparse_top_k_categorical_accuracy: 0.4548 - val_loss: 2.3231 - val_sparse_top_k_categorical_accuracy: 0.4558
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3103 - sparse_top_k_categorical_accuracy: 0.4613 - val_loss: 2.3261 - val_sparse_top_k_categorical_accuracy: 0.4547
Epoch 44/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3100 - sparse_top_k_categorical_accuracy: 0.4610 - val_loss: 2.3275 - val_sparse_top_k_categorical_accuracy: 0.4534
Epoch 45/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3099 - sparse_top_k_categorical_accuracy: 0.4618 - val_loss: 2.3270 - val_sparse_top_k_categorical_accuracy: 0.4547
Epoch 46/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3093 - sparse_top_k_categorical_accuracy: 0.4618 - val_loss: 2.3274 - val_sparse_top_k_categorical_accuracy: 0.4545
Epoch 47/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3093 - sparse_top_k_categorical_accuracy: 0.4615 - val_loss: 2.3281 - val_sparse_top_k_categorical_accuracy: 0.4533
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3012 - sparse_top_k_categorical_accuracy: 0.4659 - val_loss: 2.3349 - val_sparse_top_k_categorical_accuracy: 0.4495
Epoch 86/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3013 - sparse_top_k_categorical_accuracy: 0.4652 - val_loss: 2.3323 - val_sparse_top_k_categorical_accuracy: 0.4501
Epoch 87/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3014 - sparse_top_k_categorical_accuracy: 0.4652 - val_loss: 2.3323 - val_sparse_top_k_categorical_accuracy: 0.4511
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3013 - sparse_top_k_categorical_accuracy: 0.4654 - val_loss: 2.3331 - val_sparse_top_k_categorical_accuracy: 0.4503
Epoch 89/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3012 - sparse_top_k_categorical_accuracy: 0.4655 - val_loss: 2.3335 - val_sparse_top_k_categorical_accuracy: 0.4507
Epoch 90/100
1427/14

In [28]:
n_embeding=20

model_wo_D4 = tf.keras.models.Sequential()
model_wo_D4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model_wo_D4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D4] )
model_fitted_wo_D4 = model_wo_D4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D4 = model_wo_D4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3327 - sparse_top_k_categorical_accuracy: 0.4488 - val_loss: 2.3314 - val_sparse_top_k_categorical_accuracy: 0.4543
Epoch 2/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3260 - sparse_top_k_categorical_accuracy: 0.4534 - val_loss: 2.3252 - val_sparse_top_k_categorical_accuracy: 0.4552
Epoch 3/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3242 - sparse_top_k_categorical_accuracy: 0.4541 - val_loss: 2.3249 - val_sparse_top_k_categorical_accuracy: 0.4542
Epoch 4/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3226 - sparse_top_k_categorical_accuracy: 0.4553 - val_loss: 2.3240 - val_sparse_top_k_categorical_accuracy: 0.4551
Epoch 5/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3214 - sparse_top_k_categorical_accuracy: 0.4551 - val_loss: 2.3235 - val_sparse_top_k_categorical_accuracy: 0.4559
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2828 - sparse_top_k_categorical_accuracy: 0.4731 - val_loss: 2.3453 - val_sparse_top_k_categorical_accuracy: 0.4480
Epoch 44/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2819 - sparse_top_k_categorical_accuracy: 0.4721 - val_loss: 2.3465 - val_sparse_top_k_categorical_accuracy: 0.4478
Epoch 45/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2812 - sparse_top_k_categorical_accuracy: 0.4727 - val_loss: 2.3457 - val_sparse_top_k_categorical_accuracy: 0.4477
Epoch 46/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2806 - sparse_top_k_categorical_accuracy: 0.4733 - val_loss: 2.3470 - val_sparse_top_k_categorical_accuracy: 0.4466
Epoch 47/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2800 - sparse_top_k_categorical_accuracy: 0.4727 - val_loss: 2.3514 - val_sparse_top_k_categorical_accuracy: 0.4467
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2602 - sparse_top_k_categorical_accuracy: 0.4826 - val_loss: 2.3710 - val_sparse_top_k_categorical_accuracy: 0.4421
Epoch 86/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2600 - sparse_top_k_categorical_accuracy: 0.4822 - val_loss: 2.3726 - val_sparse_top_k_categorical_accuracy: 0.4417
Epoch 87/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2592 - sparse_top_k_categorical_accuracy: 0.4824 - val_loss: 2.3734 - val_sparse_top_k_categorical_accuracy: 0.4425
Epoch 88/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2589 - sparse_top_k_categorical_accuracy: 0.4826 - val_loss: 2.3737 - val_sparse_top_k_categorical_accuracy: 0.4420
Epoch 89/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2579 - sparse_top_k_categorical_accuracy: 0.4826 - val_loss: 2.3751 - val_sparse_top_k_categorical_accuracy: 0.4404
Epoch 90/100
1427/14

In [29]:
n_embeding=20

model_wD4 = tf.keras.models.Sequential()
model_wD4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model_wD4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD4.add(tf.keras.layers.Dropout(0.25))
model_wD4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD4] )
model_fitted_wD4 = model_wD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD4 = model_wD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3337 - sparse_top_k_categorical_accuracy: 0.4484 - val_loss: 2.3271 - val_sparse_top_k_categorical_accuracy: 0.4546
Epoch 2/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3267 - sparse_top_k_categorical_accuracy: 0.4537 - val_loss: 2.3254 - val_sparse_top_k_categorical_accuracy: 0.4539
Epoch 3/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3248 - sparse_top_k_categorical_accuracy: 0.4535 - val_loss: 2.3249 - val_sparse_top_k_categorical_accuracy: 0.4546
Epoch 4/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3235 - sparse_top_k_categorical_accuracy: 0.4543 - val_loss: 2.3235 - val_sparse_top_k_categorical_accuracy: 0.4555
Epoch 5/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3226 - sparse_top_k_categorical_accuracy: 0.4558 - val_loss: 2.3229 - val_sparse_top_k_categorical_accuracy: 0.4552
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2979 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.3338 - val_sparse_top_k_categorical_accuracy: 0.4535
Epoch 44/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2973 - sparse_top_k_categorical_accuracy: 0.4672 - val_loss: 2.3351 - val_sparse_top_k_categorical_accuracy: 0.4507
Epoch 45/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2970 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.3345 - val_sparse_top_k_categorical_accuracy: 0.4531
Epoch 46/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2965 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.3340 - val_sparse_top_k_categorical_accuracy: 0.4515
Epoch 47/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2961 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.3351 - val_sparse_top_k_categorical_accuracy: 0.4511
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2850 - sparse_top_k_categorical_accuracy: 0.4713 - val_loss: 2.3477 - val_sparse_top_k_categorical_accuracy: 0.4441
Epoch 86/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2840 - sparse_top_k_categorical_accuracy: 0.4720 - val_loss: 2.3452 - val_sparse_top_k_categorical_accuracy: 0.4485
Epoch 87/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2840 - sparse_top_k_categorical_accuracy: 0.4718 - val_loss: 2.3465 - val_sparse_top_k_categorical_accuracy: 0.4482
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2843 - sparse_top_k_categorical_accuracy: 0.4719 - val_loss: 2.3450 - val_sparse_top_k_categorical_accuracy: 0.4453
Epoch 89/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2831 - sparse_top_k_categorical_accuracy: 0.4720 - val_loss: 2.3459 - val_sparse_top_k_categorical_accuracy: 0.4483
Epoch 90/100
1427/14

In [30]:
n_embeding=20

model_wDD4 = tf.keras.models.Sequential()
model_wDD4.add(tf.keras.layers.Dense(512, activation='relu', input_dim=27))
model_wDD4.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD4] )
model_fitted_wDD4= model_wDD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD4 = model_wDD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3371 - sparse_top_k_categorical_accuracy: 0.4475 - val_loss: 2.3274 - val_sparse_top_k_categorical_accuracy: 0.4547
Epoch 2/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3290 - sparse_top_k_categorical_accuracy: 0.4527 - val_loss: 2.3270 - val_sparse_top_k_categorical_accuracy: 0.4555
Epoch 3/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3272 - sparse_top_k_categorical_accuracy: 0.4531 - val_loss: 2.3285 - val_sparse_top_k_categorical_accuracy: 0.4534
Epoch 4/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3261 - sparse_top_k_categorical_accuracy: 0.4539 - val_loss: 2.3248 - val_sparse_top_k_categorical_accuracy: 0.4539
Epoch 5/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3252 - sparse_top_k_categorical_accuracy: 0.4536 - val_loss: 2.3251 - val_sparse_top_k_categorical_accuracy: 0.4539
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3097 - sparse_top_k_categorical_accuracy: 0.4612 - val_loss: 2.3282 - val_sparse_top_k_categorical_accuracy: 0.4530
Epoch 44/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3094 - sparse_top_k_categorical_accuracy: 0.4610 - val_loss: 2.3271 - val_sparse_top_k_categorical_accuracy: 0.4516
Epoch 45/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3091 - sparse_top_k_categorical_accuracy: 0.4604 - val_loss: 2.3281 - val_sparse_top_k_categorical_accuracy: 0.4530
Epoch 46/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3089 - sparse_top_k_categorical_accuracy: 0.4607 - val_loss: 2.3263 - val_sparse_top_k_categorical_accuracy: 0.4526
Epoch 47/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.3086 - sparse_top_k_categorical_accuracy: 0.4621 - val_loss: 2.3269 - val_sparse_top_k_categorical_accuracy: 0.4534
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3013 - sparse_top_k_categorical_accuracy: 0.4648 - val_loss: 2.3330 - val_sparse_top_k_categorical_accuracy: 0.4514
Epoch 86/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3013 - sparse_top_k_categorical_accuracy: 0.4642 - val_loss: 2.3322 - val_sparse_top_k_categorical_accuracy: 0.4511
Epoch 87/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3013 - sparse_top_k_categorical_accuracy: 0.4654 - val_loss: 2.3314 - val_sparse_top_k_categorical_accuracy: 0.4501
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3008 - sparse_top_k_categorical_accuracy: 0.4648 - val_loss: 2.3325 - val_sparse_top_k_categorical_accuracy: 0.4525
Epoch 89/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3008 - sparse_top_k_categorical_accuracy: 0.4651 - val_loss: 2.3323 - val_sparse_top_k_categorical_accuracy: 0.4525
Epoch 90/100
1427/14

In [31]:
print(len(xValidation))
mapping = dict(zip(le_d.classes_, range(len(le_d.classes_))))
print(mapping)
predict = model_wDD.predict_classes(xValidation)
unique, counts = np.unique(predict, return_counts=True)
print (np.asarray((unique, counts)).T)
unique, counts = np.unique(yValidation, return_counts=True)

print (np.asarray((unique, counts)).T)

60846
{'A1': 0, 'A15': 1, 'A7': 2, 'B2': 3, 'B3': 4, 'C11': 5, 'C6': 6, 'D14': 7, 'D4': 8, 'E13': 9, 'E18': 10, 'E5': 11}
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[    0  3012]
 [    2    31]
 [    3 28359]
 [    4  2785]
 [    5 11578]
 [    6   453]
 [    7    99]
 [    8 14431]
 [    9    19]
 [   10    64]
 [   11    15]]
[[   0 6637]
 [   1 1199]
 [   2 2446]
 [   3 9265]
 [   4 6963]
 [   5 8124]
 [   6 4831]
 [   7 3846]
 [   8 7939]
 [   9 3412]
 [  10 3583]
 [  11 2601]]


In [32]:
print(len(xValidation))
mapping = dict(zip(le_d.classes_, range(len(le_d.classes_))))
print(mapping)
predict = model_wD.predict_classes(xValidation)
unique, counts = np.unique(predict, return_counts=True)
print (np.asarray((unique, counts)).T)
unique, counts = np.unique(yValidation, return_counts=True)

print (np.asarray((unique, counts)).T)

60846
{'A1': 0, 'A15': 1, 'A7': 2, 'B2': 3, 'B3': 4, 'C11': 5, 'C6': 6, 'D14': 7, 'D4': 8, 'E13': 9, 'E18': 10, 'E5': 11}
[[    0  3955]
 [    1     8]
 [    2    27]
 [    3 21947]
 [    4  5713]
 [    5 17341]
 [    6   518]
 [    7    59]
 [    8 11066]
 [    9    39]
 [   10   117]
 [   11    56]]
[[   0 6637]
 [   1 1199]
 [   2 2446]
 [   3 9265]
 [   4 6963]
 [   5 8124]
 [   6 4831]
 [   7 3846]
 [   8 7939]
 [   9 3412]
 [  10 3583]
 [  11 2601]]
